<a href="https://colab.research.google.com/github/papagorgio23/FantasyFootball/blob/master/Fantasy_Football_Lineup_Optimizer_Complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![A.I. Sports](https://raw.githubusercontent.com/papagorgio23/Python101/master/Logo.png)

www.AISportsFirm.com

# Daily Fantasy Football Lineup Optimizer

![Draft Kings](https://raw.githubusercontent.com/papagorgio23/Python101/master/DraftKings.png)

We are going to tap into the Draft Kings API to access player prices and average player scores. 

We are then going to use this information to create optimal lineups. 

*   Optimial lineup = Combination of players that will score the most points


## Linear Programming (The Math)

The basic framework of linear/integer programming is as follows:


*   There are **Decision Variables**
    *  These will be the variables that we can select (i.e. which player to have in our lineup)
*   There is an **Objective Function** to either Maximize or Minimize
    * Our goal will be to Maximize our point total
*   There are **Constraints** we must abide by
    * Salary, Number of Players, Players at specific positions, non-negativity




## Definitions:

<br>

**Index Sets:**

Let ***I*** be the set of positions, indexed by *i*

Let ***J*** be the set of players, indexed by *j*

<br>

**Variables:**

Let ***P*** be the projected points

Let ***C*** be the cost of the player

<br>

**Other Variables:**

Let ***n*** be the total number of positions

Let ***m*** be the total number of players


***

<br>

$x_{ij} = Player$

<br>

$P_{ij} = Points\ Player\ Scores$

<br>

$C_{ij} = Cost\ of\ Player$

<br>

***

<br>


#### implicit Form:


$$maximize\ \ \ \ \ \ z = \sum_{i=1}^n\sum_{j=1}^m P_{ij} * x_{ij} $$

<br>

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ subject\ to\ \ \ \ \ \sum_{i=1}^n\sum_{j=1}^m C_{ij} * x_{ij} \leq Max\ Salary$$


$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij} =1, \ \ for\ \ i = QB$$


$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij}  =2, \ \ for\ \ i = RB$$

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij} =3, \ \ for\ \ i = WR$$

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij} =1, \ \ for\ \ i = TE$$

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij} =1, \ \ for\ \ i = FLEX$$

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij} =1, \ \ for\ \ i = DST$$

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{i=1}^n\sum_{j=1}^m x_{ij} = 9$$

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ x_{ij} \geq 0,\ \forall \ i,\ j$$

<br>

***

## Daily Fantasy Football Rules

$50,000 to spend on your lineup 

https://www.draftkings.com/help/rules/nfl

![Offense](https://raw.githubusercontent.com/papagorgio23/Python101/master/DFS%20Offense%20Scoring.png)

![Defense](https://raw.githubusercontent.com/papagorgio23/Python101/master/DFS%20Defense%20Scoring.png)

# Python Code

### Load Libraries

In [0]:
!pip install pulp

import urllib, json
import pandas as pd
import re

from pulp import *

## Get Data

### DraftKings API

DraftKings API is undocumented... (don't report it to ICE)... 

If we enter the correct weekID, it will return a list of player prices and scores. 

We will need to make a few manipulations to get it into a form that we can use for our model.

### API URL

https://api.draftkings.com/draftgroups/v1/draftgroups/21727/draftables?format=json

In [0]:
# Week ID
weekID = '21727'

In [0]:
# input the week ID into the rest of the url
URL = "https://api.draftkings.com/draftgroups/v1/draftgroups/" + weekID + "/draftables?format=json"
URL

'https://api.draftkings.com/draftgroups/v1/draftgroups/21727/draftables?format=json'

### Get Data

In [0]:
# connect to API
response = urllib.request.urlopen(URL)

# extract data
data = json.loads(response.read())


In [0]:
# put it into a dataframe
current = pd.DataFrame.from_dict(data["draftables"])

## Clean Data


This data is in a horrible format. 

*   There are Dictionaries nested in column
*   Dictionary within a List nested in column
*   Duplicates
*   Irrelevant columns



In [0]:
# View Data
current.head()

,competition,displayName,draftAlerts,draftStatAttributes,draftableId,firstName,isDisabled,isSwappable,lastName,newsStatus,playerAttributes,playerGameAttributes,playerGameHash,playerId,playerImage160,playerImage50,playerImage65,playerImageFull,position,rosterSlotId,salary,shortName,status,teamAbbreviation,teamId
0,"{'competitionId': 5523569, 'name': 'NO @ NYG',...",Alvin Kamara,[],"[{'id': 90, 'value': '36.3', 'sortValue': '36....",11349545,Alvin,False,False,Kamara,Recent,[],[],750846-5523569,750846,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_re...,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_50...,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_65...,https://d327rxwuxd0q0c.cloudfront.net/nfl/play...,RB,67,9600,A. Kamara,None,NO,350
1,"{'competitionId': 5523569, 'name': 'NO @ NYG',...",Alvin Kamara,[],"[{'id': 90, 'value': '36.3', 'sortValue': '36....",11349546,Alvin,False,False,Kamara,Recent,[],[],750846-5523569,750846,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_re...,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_50...,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_65...,https://d327rxwuxd0q0c.cloudfront.net/nfl/play...,RB,70,9600,A. Kamara,None,NO,350
2,"{'competitionId': 5523569, 'name': 'NO @ NYG',...",Michael Thomas,[],"[{'id': 90, 'value': '27.1', 'sortValue': '27....",11349345,Michael,False,False,Thomas,Recent,[],[],653699-5523569,653699,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_re...,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_50...,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_65...,https://d327rxwuxd0q0c.cloudfront.net/nfl/play...,WR,68,9100,M. Thomas,None,NO,350
3,"{'competitionId': 5523569, 'name': 'NO @ NYG',...",Michael Thomas,[],"[{'id': 90, 'value': '27.1', 'sortValue': '27....",11349346,Michael,False,False,Thomas,Recent,[],[],653699-5523569,653699,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_re...,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_50...,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_65...,https://d327rxwuxd0q0c.cloudfront.net/nfl/play...,WR,70,9100,M. Thomas,None,NO,350
4,"{'competitionId': 5523581, 'name': 'BAL @ PIT'...",Antonio Brown,[],"[{'id': 90, 'value': '18.6', 'sortValue': '18....",11348973,Antonio,False,False,Brown,Recent,[],[],406214-5523581,406214,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_re...,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_50...,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_65...,https://d327rxwuxd0q0c.cloudfront.net/nfl/play...,WR,68,8800,A. Brown,None,PIT,356


### Remove Duplicates

In [0]:
# dropping duplicte Names 
current = current.drop_duplicates(subset = 'displayName', keep = 'first') 

### Remove Injured Players

In [0]:
# status counts
current['status'].value_counts()


None    430
Q        47
IR       17
O        16
Name: status, dtype: int64

In [0]:
# Remove players that are out or questionable
current = current[current.status == "None"]

In [0]:
# view data
current.head()

,competition,displayName,draftAlerts,draftStatAttributes,draftableId,firstName,isDisabled,isSwappable,lastName,newsStatus,playerAttributes,playerGameAttributes,playerGameHash,playerId,playerImage160,playerImage50,playerImage65,playerImageFull,position,rosterSlotId,salary,shortName,status,teamAbbreviation,teamId
0,"{'competitionId': 5523569, 'name': 'NO @ NYG',...",Alvin Kamara,[],"[{'id': 90, 'value': '36.3', 'sortValue': '36....",11349545,Alvin,False,False,Kamara,Recent,[],[],750846-5523569,750846,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_re...,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_50...,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_65...,https://d327rxwuxd0q0c.cloudfront.net/nfl/play...,RB,67,9600,A. Kamara,None,NO,350
2,"{'competitionId': 5523569, 'name': 'NO @ NYG',...",Michael Thomas,[],"[{'id': 90, 'value': '27.1', 'sortValue': '27....",11349345,Michael,False,False,Thomas,Recent,[],[],653699-5523569,653699,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_re...,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_50...,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_65...,https://d327rxwuxd0q0c.cloudfront.net/nfl/play...,WR,68,9100,M. Thomas,None,NO,350
4,"{'competitionId': 5523581, 'name': 'BAL @ PIT'...",Antonio Brown,[],"[{'id': 90, 'value': '18.6', 'sortValue': '18....",11348973,Antonio,False,False,Brown,Recent,[],[],406214-5523581,406214,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_re...,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_50...,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_65...,https://d327rxwuxd0q0c.cloudfront.net/nfl/play...,WR,68,8800,A. Brown,None,PIT,356
6,"{'competitionId': 5523569, 'name': 'NO @ NYG',...",Odell Beckham Jr.,[],"[{'id': 90, 'value': '17.8', 'sortValue': '17....",11349201,Odell,False,False,Beckham Jr.,Recent,[],[],589984-5523569,589984,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_re...,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_50...,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_65...,https://d327rxwuxd0q0c.cloudfront.net/nfl/play...,WR,68,8700,O. Beckham Jr.,None,NYG,351
10,"{'competitionId': 5523526, 'name': 'HOU @ IND'...",DeAndre Hopkins,[],"[{'id': 90, 'value': '22.9', 'sortValue': '22....",11349188,DeAndre,False,False,Hopkins,Recent,[],[],560241-5523526,560241,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_re...,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_50...,https://d327rxwuxd0q0c.cloudfront.net/m/nfl_65...,https://d327rxwuxd0q0c.cloudfront.net/nfl/play...,WR,68,8400,D. Hopkins,None,HOU,325


### Extract Points

"draftStatAttributes" contains the average fantasy points for each player. However, it is nested in a very annoying way.


We need to create a function that will 

In [0]:
# examine how draftStatAttributes is formatted
current['draftStatAttributes'][2]

[{'id': 90, 'sortValue': '27.1', 'value': '27.1'},
 {'id': -2, 'quality': 'Low', 'sortValue': '2', 'value': '2nd'}]

In [0]:
# Create function to extract average points
def get_float(l, key):
    """ Returns first float value from a list of dictionaries based on key. Defaults to 0.0 """
    for d in l:
        try:
            return float(d.get(key))
        except:
            pass
    return 0.0

In [0]:
# use function on dataset
current["points"] = [get_float(x, "value") for x in current.draftStatAttributes]

### Select Columns

We only need the player's position, displayName, salary, and points

In [0]:
# subset our data and name it availables
availables = current[["position", "displayName", "salary", "points"]]

# view top 5 players
availables.head()

,position,displayName,salary,points
0,RB,Alvin Kamara,9600,36.3
2,WR,Michael Thomas,9100,27.1
4,WR,Antonio Brown,8800,18.6
6,WR,Odell Beckham Jr.,8700,17.8
10,WR,DeAndre Hopkins,8400,22.9


In [0]:
# reset the index numbers
availables = availables.reset_index(drop=True)

# view top 5 players
availables.head()

,position,displayName,salary,points
0,RB,Alvin Kamara,9600,36.3
1,WR,Michael Thomas,9100,27.1
2,WR,Antonio Brown,8800,18.6
3,WR,Odell Beckham Jr.,8700,17.8
4,WR,DeAndre Hopkins,8400,22.9


In [0]:
# See how many players are available at each position
availables["position"].value_counts()

WR     143
RB     105
TE      87
QB      67
DST     28
Name: position, dtype: int64

In [0]:
# See top 15 priced QBs
availables[availables["position"]=="QB"].sort_values(by = 'points', ascending=False).head(15)

,position,displayName,salary,points
19,QB,Patrick Mahomes,6800,31.1
30,QB,Matt Ryan,6100,29.2
29,QB,Ryan Fitzpatrick,6200,28.6
23,QB,Drew Brees,6600,26.1
27,QB,Deshaun Watson,6300,26.0
32,QB,Ben Roethlisberger,6000,25.7
25,QB,Philip Rivers,6500,23.9
57,QB,Andy Dalton,5400,23.5
50,QB,Andrew Luck,5500,22.6
62,QB,Blake Bortles,5200,22.0


# Define The Problem

## Using PuLP (Random Example)

There was a natural disaster, Earthquake that caused a Tsunami and first aid is being shipped in to help the situation.

There are 2 distribution centers (DC1 and DC2) that can ship supplies to either the earthquake location or the tsunami location. Each location needs a set number of supplies. 

What is the shortest distance needed to ship all the needed supplies?

In [0]:
# initiate the problem statement
prob = LpProblem("Natural Disaster Relief", LpMinimize)

# create the variables
y1 = LpVariable("First Aid shipments from DC1 to Earthquake", 0, None, LpInteger)
y2 = LpVariable("First Aid shipments from DC1 to Tsunami", 0, None, LpInteger)
y3 = LpVariable("First Aid shipments from DC2 to Earthquake", 0, None, LpInteger)
y4 = LpVariable("First Aid shipments from DC2 to Tsunami", 0, None, LpInteger)

# create the objective function 
prob += 240*y1 + 320*y2 + 440*y3 + 180*y4

# create the constraints
prob += y1 + y2 <= 10, "DC1 has 10 shipping containers of First Aid supplies"
prob += y3 + y4 <= 4, "DC2 has 4 shipping containers of First Aid supplies"
prob += y1 + y3 >= 8, "8 containers of supplies needed for Earthquake Relief"
prob += y2 + y4 >= 6, "6 containers of supplies needed for Tsunami Relief"

# solve the problem
prob.solve()

# print each variable with its optimal value
for v in prob.variables():
  print(v.name, "=", v.varValue)

# print the optimized objective function value 
print("\nMinimized total transit distance = ", value(prob.objective),"miles")

## Prepare for LP formula


The data needs to be transformed slightly differently for PuLP to work with it.


<br>

$P_{ij} = Points\ Player\ Scores$

<br>

$C_{ij} = Cost\ of\ Player$

<br>

In [0]:
# create 2 dictionaries: one containing the salaries of each player, the other containing the points of each player

# Cost of Player (C_ij)
salaries = {} 

# Points of Player (P_ij)
points = {} 

# loop through each position and appened player name, salary, points to correct dictionary
for pos in availables["position"].unique():
    available_pos = availables[availables["position"] == pos]
    salary = list(available_pos[["displayName","salary"]].set_index("displayName").to_dict().values())[0]
    point = list(available_pos[["displayName","points"]].set_index("displayName").to_dict().values())[0]
    salaries[pos] = salary
    points[pos] = point

## Position Constraints

<br>

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij} =1, \ \ for\ \ i = QB$$


$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij}  =2, \ \ for\ \ i = RB$$

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij} =3, \ \ for\ \ i = WR$$

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij} =1, \ \ for\ \ i = TE$$

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij} =1, \ \ for\ \ i = FLEX$$

$$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \sum_{j=1}^m x_{ij} =1, \ \ for\ \ i = DST$$

<br>

Flex position can be either a RB, WR, or TE. 

We will run 3 iterations with the flex being each on of these positions.

In [0]:
# Position Constraints
pos_num_available = {
    "QB": 1,
    "RB": 2,
    "WR": 3,
    "TE": 2,
    "DST": 1
}

In [0]:
# Max Salary limit
SALARY_CAP = 50000

## Create Player Variables

<br>

$x_{ij} = Player$


<br>

In [0]:
# Create the variables
_vars = {k: LpVariable.dict(k, v, cat="Binary") for k, v in points.items()}

## Solve Problem

In [0]:
# initialize the problem statement (Maximize)
prob = LpProblem("Fantasy Football Lineup", LpMaximize)

# initialize lists to hold constraints
score = []
salary = []

# Setting up the reward
for k, v in _vars.items():
    salary += lpSum([salaries[k][i] * _vars[k][i] for i in v])
    score += lpSum([points[k][i] * _vars[k][i] for i in v])
    prob += lpSum([_vars[k][i] for i in v]) <= pos_num_available[k]
    

# Salary Constraint
prob += lpSum(salary) <= SALARY_CAP
    
# Objective Function
prob += lpSum(score)

In [0]:
prob.solve()

1

In [0]:
# my beautiful function to display the results of our LP
def summary(prob):
    div = '\n-----------------------------------------\n'
    print("\n-----------------------------------------")
    print("Linear Programming Optimization Solution:")
    print("-----------------------------------------\n")
    print("Variables:\n")
    score = str(prob.objective)
    constraints = [str(const) for const in prob.constraints.values()]
    for v in prob.variables():
        score = score.replace(v.name, str(v.varValue))
        constraints = [const.replace(v.name, str(v.varValue)) for const in constraints]
        if v.varValue != 0:
            print(v.name, "=", v.varValue)
    print(div)
    print("Salary:\n")
    for constraint in constraints:
        constraint_pretty = " + ".join(re.findall("[0-9\.]*\*1.0", constraint))
        if constraint_pretty != "":
            print("{} = {}".format(constraint_pretty, eval(constraint_pretty)))
    print(div)
    print("Score:\n")
    score_pretty = " + ".join(re.findall("[0-9\.]+\*1.0", score))
    print("{} = {}".format(score_pretty, eval(score)))
    print(div)
    print("Total Cost:  ${}".format(round(eval(constraint_pretty))))
    print("Total Points: {}\n".format(round(eval(score))))

In [0]:
summary(prob)


-----------------------------------------
Linear Programming Optimization Solution:
-----------------------------------------

Variables:

DST_Bears_ = 1.0
QB_Patrick_Mahomes = 1.0
RB_Alvin_Kamara = 1.0
RB_Melvin_Gordon_III = 1.0
TE_George_Kittle = 1.0
TE_Jared_Cook = 1.0
WR_Golden_Tate = 1.0
WR_Keke_Coutee = 1.0
WR_Tyler_Boyd = 1.0

-----------------------------------------

Salary:

2600*1.0 + 6800*1.0 + 9600*1.0 + 8300*1.0 + 4200*1.0 + 4100*1.0 + 6600*1.0 + 3000*1.0 + 4600*1.0 = 49800.0

-----------------------------------------

Score:

14.5*1.0 + 31.1*1.0 + 36.3*1.0 + 27.9*1.0 + 15.2*1.0 + 20.3*1.0 + 23.0*1.0 + 24.7*1.0 + 19.9*1.0 = 212.9

-----------------------------------------

Total Cost:  $49800
Total Points: 213



## Flex Postion = RB

In [0]:
# salary and points
salaries = {} 
points = {} 
for pos in availables["position"].unique():
    available_pos = availables[availables["position"] == pos]
    salary = list(available_pos[["displayName","salary"]].set_index("displayName").to_dict().values())[0]
    point = list(available_pos[["displayName","points"]].set_index("displayName").to_dict().values())[0]
    salaries[pos] = salary
    points[pos] = point

# Max Salary limit
SALARY_CAP = 50000


# Position Constraints
pos_num_available = {
    "QB": 1,
    "RB": 3, # FLEX POSITION = RB
    "WR": 3, 
    "TE": 1, 
    "DST": 1
}



# initialize the problem statement (Maximize)
prob = LpProblem("Fantasy Football Lineup", LpMaximize)

# Create the variables
_vars = {k: LpVariable.dict(k, v, cat="Binary") for k, v in points.items()}

# initialize lists to hold constraints
score = []
salary = []

# Setting up the reward
for k, v in _vars.items():
    salary += lpSum([salaries[k][i] * _vars[k][i] for i in v])
    score += lpSum([points[k][i] * _vars[k][i] for i in v])
    prob += lpSum([_vars[k][i] for i in v]) <= pos_num_available[k]
    

# Salary Constraint
prob += lpSum(salary) <= SALARY_CAP
    
# Objective Function
prob += lpSum(score)

prob.solve()

summary(prob)


-----------------------------------------
Linear Programming Optimization Solution:
-----------------------------------------

Variables:

DST_Bears_ = 1.0
QB_Patrick_Mahomes = 1.0
RB_Alvin_Kamara = 1.0
RB_James_White = 1.0
RB_Melvin_Gordon_III = 1.0
TE_Jared_Cook = 1.0
WR_Calvin_Ridley = 1.0
WR_Keke_Coutee = 1.0
WR_Tyler_Boyd = 1.0

-----------------------------------------

Salary:

2600*1.0 + 6800*1.0 + 9600*1.0 + 5400*1.0 + 8300*1.0 + 4100*1.0 + 4900*1.0 + 3000*1.0 + 4600*1.0 = 49300.0

-----------------------------------------

Score:

14.5*1.0 + 31.1*1.0 + 36.3*1.0 + 19.1*1.0 + 27.9*1.0 + 20.3*1.0 + 20.4*1.0 + 24.7*1.0 + 19.9*1.0 = 214.20000000000002

-----------------------------------------

Total Cost:  $49300
Total Points: 214



## Flex Postion = WR

In [0]:
# salary and points
salaries = {} 
points = {} 
for pos in availables["position"].unique():
    available_pos = availables[availables["position"] == pos]
    salary = list(available_pos[["displayName","salary"]].set_index("displayName").to_dict().values())[0]
    point = list(available_pos[["displayName","points"]].set_index("displayName").to_dict().values())[0]
    salaries[pos] = salary
    points[pos] = point

# Max Salary limit
SALARY_CAP = 50000


# Position Constraints
pos_num_available = {
    "QB": 1,
    "RB": 2,
    "WR": 4, # FLEX POSITION = WR
    "TE": 1, 
    "DST": 1
}



# initialize the problem statement (Maximize)
prob = LpProblem("Fantasy Football Lineup", LpMaximize)

# Create the variables
_vars = {k: LpVariable.dict(k, v, cat="Binary") for k, v in points.items()}

# initialize lists to hold constraints
score = []
salary = []

# Setting up the reward
for k, v in _vars.items():
    salary += lpSum([salaries[k][i] * _vars[k][i] for i in v])
    score += lpSum([points[k][i] * _vars[k][i] for i in v])
    prob += lpSum([_vars[k][i] for i in v]) <= pos_num_available[k]
    

# Salary Constraint
prob += lpSum(salary) <= SALARY_CAP
    
# Objective Function
prob += lpSum(score)

prob.solve()

summary(prob)


-----------------------------------------
Linear Programming Optimization Solution:
-----------------------------------------

Variables:

DST_Bears_ = 1.0
QB_Patrick_Mahomes = 1.0
RB_Alvin_Kamara = 1.0
RB_Melvin_Gordon_III = 1.0
TE_Jared_Cook = 1.0
WR_Calvin_Ridley = 1.0
WR_DeSean_Jackson = 1.0
WR_Keke_Coutee = 1.0
WR_Tyler_Boyd = 1.0

-----------------------------------------

Salary:

2600*1.0 + 6800*1.0 + 9600*1.0 + 8300*1.0 + 4100*1.0 + 4900*1.0 + 5700*1.0 + 3000*1.0 + 4600*1.0 = 49600.0

-----------------------------------------

Score:

14.5*1.0 + 31.1*1.0 + 36.3*1.0 + 27.9*1.0 + 20.3*1.0 + 20.4*1.0 + 21.6*1.0 + 24.7*1.0 + 19.9*1.0 = 216.70000000000002

-----------------------------------------

Total Cost:  $49600
Total Points: 217



## Flex Postion = TE

In [0]:
# salary and points
salaries = {} 
points = {} 
for pos in availables["position"].unique():
    available_pos = availables[availables["position"] == pos]
    salary = list(available_pos[["displayName","salary"]].set_index("displayName").to_dict().values())[0]
    point = list(available_pos[["displayName","points"]].set_index("displayName").to_dict().values())[0]
    salaries[pos] = salary
    points[pos] = point

# Max Salary limit
SALARY_CAP = 50000


# Position Constraints
pos_num_available = {
    "QB": 1,
    "RB": 2,
    "WR": 3,
    "TE": 2, # FLEX POSITION = TE
    "DST": 1
}



# initialize the problem statement (Maximize)
prob = LpProblem("Fantasy Football Lineup", LpMaximize)

# Create the variables
_vars = {k: LpVariable.dict(k, v, cat="Binary") for k, v in points.items()}

# initialize lists to hold constraints
score = []
salary = []

# Setting up the reward
for k, v in _vars.items():
    salary += lpSum([salaries[k][i] * _vars[k][i] for i in v])
    score += lpSum([points[k][i] * _vars[k][i] for i in v])
    prob += lpSum([_vars[k][i] for i in v]) <= pos_num_available[k]
    

# Salary Constraint
prob += lpSum(salary) <= SALARY_CAP
    
# Objective Function
prob += lpSum(score)

prob.solve()

summary(prob)


-----------------------------------------
Linear Programming Optimization Solution:
-----------------------------------------

Variables:

DST_Bears_ = 1.0
QB_Patrick_Mahomes = 1.0
RB_Alvin_Kamara = 1.0
RB_Melvin_Gordon_III = 1.0
TE_George_Kittle = 1.0
TE_Jared_Cook = 1.0
WR_Golden_Tate = 1.0
WR_Keke_Coutee = 1.0
WR_Tyler_Boyd = 1.0

-----------------------------------------

Salary:

2600*1.0 + 6800*1.0 + 9600*1.0 + 8300*1.0 + 4200*1.0 + 4100*1.0 + 6600*1.0 + 3000*1.0 + 4600*1.0 = 49800.0

-----------------------------------------

Score:

14.5*1.0 + 31.1*1.0 + 36.3*1.0 + 27.9*1.0 + 15.2*1.0 + 20.3*1.0 + 23.0*1.0 + 24.7*1.0 + 19.9*1.0 = 212.9

-----------------------------------------

Total Cost:  $49800
Total Points: 213



# Check Results against Actuals


https://rotogrinders.com/resultsdb/site/draftkings/date/2018-09-30/sport/nfl/slate/5bb10388ed767f5afae3935e


![Results](https://raw.githubusercontent.com/papagorgio23/Python101/master/Sept%2030%20DFS%20Results.png)